In [1]:
from pattern.web import URL, DOM, plaintext, Element, extension, Crawler, DEPTH
import re
import pickle
import random
import PIL
import time
from PIL import Image

In [2]:
class Scraper():
    def save_image(self):
        pass
    def get_recipe(self):
        pass

In [3]:
class AllRecipesScraper(Scraper):
    
    def save_image(self,element,idx,basewidth = 300):
        first_rec_photo = element.by_class("rec-photo")[0]
        url = first_rec_photo.attributes.get('src','')
        if '/nophoto/nopicture' not in url:
            img_url = URL(url)
            img = Image.open(img_url)
            wpercent = (basewidth / float(img.size[0]))
            hsize = int((float(img.size[1]) * float(wpercent)))
            img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
            img.save("img/"+str(idx) + extension(img_url.page))
            return True
        else:
            return False
        
        
    def get_ingredients(self,element):
        ing_nodes = element.by_class("recipe-ingred_txt added")
        return "\n".join([plaintext(a.content) for a in ing_nodes])

    
    def get_instructions(self,element):
        instr_nodes = element.by_class("recipe-directions__list--item")
        return "\n".join([plaintext(a.content) for a in instr_nodes])
    
    def get_recipe(self,element):
        return self.get_ingredients(element)+"\n"+self.get_instructions(element)

In [4]:
class AllRecipesRandomSearch():
    def __init__(self,tried_ids = None, recipe_list = None ):
        self.scraper = AllRecipesScraper()
        if tried_ids is None:
            self.tried_ids = set()
        else:
            self.tried_ids = tried_ids
        if recipe_list is None:
            self.recipe_list = {}
        else:
            self.recipe_list = recipe_list
        self.count = 0
        
    def reset_count(self):
        self.count = 0
        
    def new_id(self,rec_id):
        return rec_id not in self.tried_ids
          
    def visit(self,rec_id):
        url = URL("http://allrecipes.com/recipe/"+str(rec_id))
        try:
            source = url.download(cached=True)
            self.scrape(source, rec_id)
        except Exception as detail:
            print 'Unable to Scrape:', rec_id
            self.tried_ids.add(rec_id)
            
    def scrape(self,source,rec_id):
        print("scraping", rec_id)
        element = Element(source)
        recipe = self.scraper.get_recipe(element)
        if "Top pizza crust with cheese. Bake crust according to package directions. Meanwhile, in a 12-inch skillet" not in recipe:
            saved = self.scraper.save_image(element, rec_id)
            if saved:
                self.recipe_list[rec_id]=recipe
            else:
                print "Image less recipe "
        else:
            print "Random Cheese page "
        self.tried_ids.add(rec_id)
        self.count += 1

In [ ]:
tried_ids = pickle.load(open("tried_ids_final.p","rb"))
recipe_list = pickle.load(open("recipes_final.p","rb"))

print len(tried_ids)

search = AllRecipesRandomSearch(tried_ids,recipe_list)
limit =3000

while search.count < limit:
    if search.count%100 ==0:
        print "Check point Iter: %s"%search.count
        pickle.dump( search.recipe_list, open( "recipes"+str(search.count/500)+".p", "wb" ) )
        pickle.dump( search.tried_ids, open( "tried_ids"+str(search.count/500)+".p", "wb" ) )
    rec_id = random.randint(100000,200000)
    if search.new_id(rec_id):
        search.visit(rec_id)
    time.sleep(1)

print "Saving the final work"
pickle.dump( search.recipe_list, open( "recipes_final.p", "wb" ) )
pickle.dump( search.tried_ids, open( "tried_ids_final.p", "wb" ) )

9333
Check point Iter: 0
('scraping', 119073)
Image less recipe 
('scraping', 179837)
Image less recipe 
('scraping', 124157)
Image less recipe 
('scraping', 106749)
Image less recipe 
('scraping', 188869)
Random Cheese page 
('scraping', 155330)
Random Cheese page 
('scraping', 175691)
Random Cheese page 
('scraping', 109994)
Random Cheese page 
('scraping', 125960)
Image less recipe 
('scraping', 149975)
('scraping', 148860)
Random Cheese page 
('scraping', 170916)
Random Cheese page 
('scraping', 183658)
Random Cheese page 
('scraping', 185954)
Random Cheese page 
('scraping', 168955)
Random Cheese page 
('scraping', 114072)
